<p align="center">
  <span style="color:Navy; font-size:200%; font-weight:bold; vertical-align:middle;">
    Temas Selectos: Python para Ciencias de la Tierra
  </span>
  <img src="attachment:logoencit.png" alt="ENCiT" width="150" style="vertical-align:middle; margin-left:20px;"/>
</p>
<p align="center" style="line-height:1.2;">
  <span style="color:RoyalBlue; font-size:160%;">Tema 4: Introducción al Aprendizaje Automático (Machine Learning) </span><br/>
  <span style="color:DodgerBlue; font-size:140%;"> Modelos supervisados: Árboles de desición </span><br/>
  <span style="font-size:100%;color:forestgreen"> Escuela Nacional de Ciencias de la Tierra  |  Semestre 2026-I</span>
</p>

---




## **<font color="SeaGreen"> Pronostico de un índice de sequía </font>**

A lo largo de esta unidad estaremos construyendo modelos de machine learning para hacer un pronóstico estacional de sequía para un punto en el noroeste de México (lat = 29.725, lon = -109.725).


### Paso 0:   Plantear el problema

* ¿Cuál es el fenómeno físico que quiero estudiar?
    **El comportamiento de la sequía** 
* ¿Cuál es la variable dependiente?
   El SPI (Standardized Precipitation Index) —o Índice Estandarizado de Precipitación— es un indicador estadístico usado para medir las anomalías de precipitación (lluvia) en una región y período determinados. Su objetivo principal es cuantificar las condiciones de sequía o exceso de lluvia de manera estandarizada y comparable en el tiempo y el espacio.

    **La variable dependiente sera el SPI de 3 meses (que corresponde a una sequia meteorologica)**

* ¿Es una regresión o una clasificación? 



### Paso 1:  Datos 

* Obtener información diversa (¿Necesitamos muchos datos?).

    ¿Existen datos confiables que pueda usar?
    ¿Los datos tienen variedad?

* Remover información falsa, buscar/llenar valores faltantes… 

    Si no lleno los vacíos, el molodelo los aprenderá también.

  
### Paso 2:  Variables predictoras 
* Proponer variables “predictoras” 

* **Hacer un análisis estadístico de las variables propuestas.**

* Definir las variables de entrada.


> Las variables que estan relacionadas con las sequías son aquellas que afectan al ciclo hidrológico y que pueden causar un estrés hídrico. Una alteración en estas variables detonan reacciones en cadena conocidas como procesos de retroalimentación.

Las variables que se proponen (inicialmente) para hacer los modelos, y sus procedencias, son las siguientes:


|Variable                                     | Abreviacion  | Base de datos  |
|---------------------------------------------|--------------|----------------|
|Precipitación acumulada                      | pcp          | CHIRPS         |
|Radiación de onda larga saliente             | OLR          | NCEP-NCAR      |
|SSTs en la región de El Niño 3.4             | SST          | NOAA           |
|Volumen de agua en la primera capa del suelo | swvl1        | ERA-5 Land     |
|Temperatura en la superficie del suelo       | skt          | ERA-5 Land     |
|Altura geopotencial en 600mb                 | hgt          | NCEP-NCAR      |


### Paso 3:  Dividir la base de datos en 2 o 3 subcategorías 
* 70-85% para entrenar el modelo 

* 5-15% para validar/evaluar el modelo

* Probar el modelo


### Paso 4: Implementar el/los modelos disponibles.
Entrenar los modelos puede ser una tarea sencilla, pero hay que considerar ciertos parámetros (e hiperparámetros):	
* La función de costo
* El tipo de entrenamiento 
* La tasa de aprendizaje 

### Paso 5: Paso 5: Evaluación de modelos. 
Se debe evaluar los modelos tanto con los datos de entrenamiento como con los datos de prueba. 



<div class="alert alert-success">
    <b>Pasos 2 al 5 : <b> La misión en esta clase será implementar modelos de árboles de decisión para tareas regresivas y de clasificación.
</div>


### **<font color="SeaGreen"> Importar la base de datos </font>**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.gridspec as gridspec

import sklearn
#from sklearn.neighbors import LocalOutlierFactor
#from scipy.spatial.distance import mahalanobis
#from scipy.stats import chi2
#import scipy.cluster.hierarchy as sch
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA

#from sklearn.linear_model import LinearRegression
#from sklearn.svm import LinearSVR
#from sklearn.svm import SVC

from sklearn.ensemble import ExtraTreesRegressor,ExtraTreesClassifier
from sklearn.tree import plot_tree

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
from sklearn import metrics

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report


In [ ]:
# carga de archivo csv
df=pd.read_csv('sequias_datos.csv')
df.head()

In [ ]:
plt.figure()
sns.lineplot(x='Year',y='spi', data=df, markers='o')
plt.title('Serie de Tiempo de SPI en MAM en lat=29.725,lon=-109.725', fontsize=14)
plt.xlabel('Año', fontsize=12)
plt.ylabel('SPI 3 meses', fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
VariablesNumAnalis = [ 'spi', 'sst_mean_DJF', 'pcp_DJF', 'hgt_DJF', 'olr_DJF', 'skt_DJF', 'swvl1_DJF']

df_DEF = df[VariablesNumAnalis]
df_DEF.head()

### **<font color="SeaGreen"> Paso 2:  Variables predictoras  </font>**

En las clases pasadas hemos estado haciendo el análisis multivariado de los datos. Estos nos ha permitido saber cuáles son las variables con mayor potencial de funcionar para nuestros modelos. 
¿Recuerdan cuáles son? 

![spearman.png](spearman.png)
![pca.png](pca.png)


In [ ]:
# Vamos a crear varios modelos para ver cuál es mejor. 

# Primero, vamos a separar la variable de los años 
anios = df[['Year']].copy()

# Ahora vamos a proponer el primer modelo con un solo predictor
x_prep_1 = df_DEF[['pcp_DJF','olr_DJF','swvl1_DJF']].copy()

x_prep_2 = df_DEF[['hgt_DJF', 'skt_DJF','sst_mean_DJF']].copy()

x_prep_3 = df_DEF[['pcp_DJF','hgt_DJF', 'olr_DJF']].copy()
 
# Separamos la variable dependiente
y = df[['spi']].copy()

In [ ]:
#Escalamos los datos

scaler = StandardScaler()

x_scaled_1 = scaler.fit_transform(x_prep_1)
x1_scaled_df = pd.DataFrame(x_scaled_1, columns=x_prep_1.columns)

x_scaled_2 = scaler.fit_transform(x_prep_2)
x2_scaled_df = pd.DataFrame(x_scaled_2, columns=x_prep_2.columns)

x_scaled_3 = scaler.fit_transform(x_prep_3)
x3_scaled_df = pd.DataFrame(x_scaled_3, columns=x_prep_3.columns)

y_df_anios = y.copy()
y_df_anios['Year'] = anios

### **<font color="SeaGreen"> Paso 3:  Dividir la base de datos en 2 o 3 subcategorías  </font>**


In [ ]:
# Ahora hacemos la division de los datos 

X_train1, X_test1, y_train1, y_test1 = train_test_split(x1_scaled_df, y, test_size=0.4, random_state=43)

X_train2, X_test2, y_train2, y_test2 = train_test_split(x2_scaled_df, y, test_size=0.4, random_state=43)

X_train3, X_test3, y_train3, y_test3 = train_test_split(x3_scaled_df, y, test_size=0.4, random_state=43)

### **<font color="SeaGreen"> Paso 4: Implementar el/los modelos disponibles. </font>**

Hoy vamos a usar árboles de desición para tareas regresivas y máquinas de soporte vectorial para clasificación.

Para mas informacion sobre los árboles de desición: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesRegressor.html


`sklearn.ensemble.ExtraTreesRegressor(n_estimators=100, *, criterion='squared_error', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=1.0, max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=False, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None, monotonic_cst=None)`



In [ ]:
# Definimos el modelo, en este primer caso usaremos un árbol de desición
dtr1 = ExtraTreesRegressor(random_state=100, n_estimators=6, max_depth=3)
# Entrenamos el modelo
dtr1.fit(X_train1,y_train1.values.flatten())


In [ ]:
# Predicciones con el conjunto de datos de entrenamiento
y1_pred_train = dtr1.predict(X_train1) 

# Predicciones con el conjunto de datos de prueba
y1_pred_test = dtr1.predict(X_test1)


In [ ]:
# Definimos el modelo, en este primer caso usaremos una regresion lineal 
dtr2 = ExtraTreesRegressor(random_state=100, n_estimators=6, max_depth=3)
# Entrenamos el modelo
dtr2.fit(X_train2,y_train2.values.flatten())

In [ ]:
# Predicciones con el conjunto de datos de entrenamiento
y2_pred_train = dtr2.predict(X_train2) 

# Predicciones con el conjunto de datos de prueba
y2_pred_test = dtr2.predict(X_test2)

In [ ]:
# Definimos el modelo, en este primer caso usaremos una regresion lineal 
dtr3 = ExtraTreesRegressor(random_state=100, n_estimators=6, max_depth=3)
# Entrenamos el modelo
dtr3.fit(X_train3,y_train3.values.flatten())

In [ ]:
# Predicciones con el conjunto de datos de entrenamiento
y3_pred_train = dtr3.predict(X_train3) 

# Predicciones con el conjunto de datos de prueba
y3_pred_test = dtr3.predict(X_test3)

### **<font color="SeaGreen"> Paso 5: Evaluación de modelos. </font>**

In [ ]:
#Creamos un DataFrame para ir guardando las evaluaciones de los modelos

perf_index={'predictores':[],'R2':[],'MSE':[]}
ML_method=[]
df_pi=pd.DataFrame(perf_index, index=ML_method)
df_pi = df_pi.rename_axis('ML methods')



In [ ]:
y1_pred_train_df = pd.DataFrame(y1_pred_train,columns=['spi_train_predicted'])
y1_pred_test_df = pd.DataFrame(y1_pred_test, columns=['spi_test_predicted'])

#Evaluacion con el conjunto de datos de entrenamiento 
r2_score_train1 = metrics.r2_score(y_train1,y1_pred_train_df)
mean_squared_error_train1 = metrics.mean_squared_error(y_train1,y1_pred_train_df)
row_list=[', '.join(x1_scaled_df.columns),r2_score_train1,mean_squared_error_train1]
df_pi.loc['DT 1']=row_list

#Evaluacion con el conjunto de datos de prueba
r2_score_test1 = metrics.r2_score(y_test1,y1_pred_test_df)
mean_squared_error_test1 = metrics.mean_squared_error(y_test1,y1_pred_test_df)

row_list=[', '.join(x_prep_1.columns),r2_score_test1,mean_squared_error_test1]
df_pi.loc['DT 1']=row_list
print(df_pi)


In [ ]:
df_predicted_vs_observed_train1 = y1_pred_train_df.copy()
df_predicted_vs_observed_train1['spi_observed'] = y_train1.copy()

plt.figure()
sns.scatterplot(x='spi_train_predicted', y='spi_observed', data = df_predicted_vs_observed_train1)
plt.plot([df_predicted_vs_observed_train1.min().min(), df_predicted_vs_observed_train1.max().max()],
         [df_predicted_vs_observed_train1.min().min(), df_predicted_vs_observed_train1.max().max()],
         linestyle='--', color='red')
plt.xlabel('spi predicho')
plt.ylabel('spi observado')
plt.title('SPI predicho vs observado con datos de entrenamiento y pcp_DJF, olr_DJF, swvl1_DJF')
plt.tight_layout()
plt.grid(True)
plt.show()

In [ ]:
df_predicted_vs_observed_test1 = y1_pred_test_df.copy()
df_predicted_vs_observed_test1['spi_observed'] = y_test1.copy()

plt.figure()
sns.scatterplot(x='spi_test_predicted', y='spi_observed', data = df_predicted_vs_observed_test1)
plt.plot([df_predicted_vs_observed_test1.min().min(), df_predicted_vs_observed_test1.max().max()],
         [df_predicted_vs_observed_test1.min().min(), df_predicted_vs_observed_test1.max().max()],
         linestyle='--', color='red')
plt.xlabel('spi predicho')
plt.ylabel('spi observado')
plt.title('SPI predicho vs observado con datos de prueba y pcp_DJF, olr_DJF, swvl1_DJF')
plt.tight_layout()
plt.grid(True)
plt.show()

In [ ]:
# Combinar predicciones con los años correspondientes
df_train_pred_rl1 = pd.DataFrame({'Year': X_train1.index.map(lambda i: y_df_anios.loc[i, 'Year']),
                              'spi_predicted': y1_pred_train_df.squeeze()})

df_test_pred_rl1 = pd.DataFrame({'Year': X_test1.index.map(lambda i: y_df_anios.loc[i, 'Year']),
                             'spi_predicted': y1_pred_test_df.squeeze()})

# Unir todo y ordenar cronológicamente
df_pred_total_rn1 = pd.concat([df_train_pred_rl1, df_test_pred_rl1])
df_pred_total_rn1 = df_pred_total_rn1.sort_values('Year').reset_index(drop=True)


plt.figure(figsize=(10,5))
plt.plot(df['Year'], df['spi'], label='Observado', marker='o')
#sns.lineplot(x='Year',y='spi', data=df, markers='o')
plt.plot(df_pred_total_rn1['Year'], df_pred_total_rn1['spi_predicted'], label='Predicho AD pcp_DJF, olr_DJF, swvl1_DJF', marker='s')
plt.legend()
plt.xlabel('Año')
plt.ylabel('SPI')
plt.title('Serie temporal observada vs predicha')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plot_tree(dtr1.estimators_[0],
               feature_names = x1_scaled_df.columns,
               filled=True,
               rounded=True)
plt.show()

In [ ]:
y2_pred_train_df = pd.DataFrame(y2_pred_train,columns=['spi_train_predicted'])
y2_pred_test_df = pd.DataFrame(y2_pred_test, columns=['spi_test_predicted'])

#Evaluacion con el conjunto de datos de entrenamiento 
r2_score_train2 = metrics.r2_score(y_train2,y2_pred_train_df)
mean_squared_error_train2 = metrics.mean_squared_error(y_train2,y2_pred_train_df)
row_list=[', '.join(x_prep_2.columns),r2_score_train2,mean_squared_error_train2]
df_pi.loc['DT 2']=row_list

#Evaluacion con el conjunto de datos de prueba
r2_score_test2 = metrics.r2_score(y_test2,y2_pred_test_df)
mean_squared_error_test2 = metrics.mean_squared_error(y_test2,y2_pred_test_df)

row_list=[', '.join(x_prep_2.columns),r2_score_test2,mean_squared_error_test2]
df_pi.loc['DT 2']=row_list
print(df_pi)

In [ ]:
df_predicted_vs_observed_train2 = y2_pred_train_df.copy()
df_predicted_vs_observed_train2['spi_observed'] = y_train2.copy()

plt.figure()
sns.scatterplot(x='spi_train_predicted', y='spi_observed', data = df_predicted_vs_observed_train2)
plt.plot([df_predicted_vs_observed_train2.min().min(), df_predicted_vs_observed_train2.max().max()],
         [df_predicted_vs_observed_train2.min().min(), df_predicted_vs_observed_train2.max().max()],
         linestyle='--', color='red')
plt.xlabel('spi predicho')
plt.ylabel('spi observado')
plt.title('SPI predicho vs observado con datos de entrenamiento y hgt_DJF, skt_DJF, sst_mean_DJF')
plt.tight_layout()
plt.grid(True)
plt.show()

In [ ]:
df_predicted_vs_observed_test2 = y2_pred_test_df.copy()
df_predicted_vs_observed_test2['spi_observed'] = y_test2.copy()

plt.figure()
sns.scatterplot(x='spi_test_predicted', y='spi_observed', data = df_predicted_vs_observed_test2)
plt.plot([df_predicted_vs_observed_test2.min().min(), df_predicted_vs_observed_test2.max().max()],
         [df_predicted_vs_observed_test2.min().min(), df_predicted_vs_observed_test2.max().max()],
         linestyle='--', color='red')
plt.xlabel('spi predicho')
plt.ylabel('spi observado')
plt.title('SPI predicho vs observado con datos de prueba y hgt_DJF, skt_DJF, sst_mean_DJF')
plt.tight_layout()
plt.grid(True)
plt.show()

In [ ]:
# Combinar predicciones con los años correspondientes
df_train_pred_rl2 = pd.DataFrame({'Year': X_train2.index.map(lambda i: y_df_anios.loc[i, 'Year']),
                              'spi_predicted': y2_pred_train_df.squeeze()})

df_test_pred_rl2 = pd.DataFrame({'Year': X_test2.index.map(lambda i: y_df_anios.loc[i, 'Year']),
                             'spi_predicted': y2_pred_test_df.squeeze()})

# Unir todo y ordenar cronológicamente
df_pred_total_rn2 = pd.concat([df_train_pred_rl2, df_test_pred_rl2])
df_pred_total_rn2 = df_pred_total_rn2.sort_values('Year').reset_index(drop=True)


plt.figure(figsize=(10,5))
plt.plot(df['Year'], df['spi'], label='Observado', marker='o')
#sns.lineplot(x='Year',y='spi', data=df, markers='o')
plt.plot(df_pred_total_rn1['Year'], df_pred_total_rn1['spi_predicted'], label='Predicho AD pcp_DJF, olr_DJF, swvl1_DJF', marker='s')
plt.plot(df_pred_total_rn2['Year'], df_pred_total_rn2['spi_predicted'], label='Predicho AD hgt_DJF, skt_DJF, sst_mean_DJF', marker='s')
plt.legend()
plt.xlabel('Año')
plt.ylabel('SPI')
plt.title('Serie temporal observada vs predicha')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plot_tree(dtr2.estimators_[0],
               feature_names = x2_scaled_df.columns,
               filled=True,
               rounded=True)
plt.show()

In [ ]:
y3_pred_train_df = pd.DataFrame(y3_pred_train,columns=['spi_train_predicted'])
y3_pred_test_df = pd.DataFrame(y3_pred_test, columns=['spi_test_predicted'])

#Evaluacion con el conjunto de datos de entrenamiento 
r2_score_train3 = metrics.r2_score(y_train3,y3_pred_train_df)
mean_squared_error_train3 = metrics.mean_squared_error(y_train3,y3_pred_train_df)
row_list=[', '.join(x_prep_3.columns),r2_score_train3,mean_squared_error_train3]
df_pi.loc['DT 3']=row_list

#Evaluacion con el conjunto de datos de prueba
r2_score_test3 = metrics.r2_score(y_test3,y3_pred_test_df)
mean_squared_error_test3 = metrics.mean_squared_error(y_test3,y3_pred_test_df)

row_list=[', '.join(x_prep_3.columns),r2_score_test3,mean_squared_error_test3]
df_pi.loc['DT 3']=row_list
print(df_pi)

In [ ]:
df_predicted_vs_observed_train3 = y3_pred_train_df.copy()
df_predicted_vs_observed_train3['spi_observed'] = y_train3.copy()

plt.figure()
sns.scatterplot(x='spi_train_predicted', y='spi_observed', data = df_predicted_vs_observed_train3)
plt.plot([df_predicted_vs_observed_train3.min().min(), df_predicted_vs_observed_train3.max().max()],
         [df_predicted_vs_observed_train3.min().min(), df_predicted_vs_observed_train3.max().max()],
         linestyle='--', color='red')
plt.xlabel('spi predicho')
plt.ylabel('spi observado')
plt.title('SPI predicho vs observado con datos de entrenamiento y pcp_DJF, hgt_DJF, olr_DJF')
plt.tight_layout()
plt.grid(True)
plt.show()

In [ ]:
df_predicted_vs_observed_test3 = y3_pred_test_df.copy()
df_predicted_vs_observed_test3['spi_observed'] = y_test3.copy()

plt.figure()
sns.scatterplot(x='spi_test_predicted', y='spi_observed', data = df_predicted_vs_observed_test3)
plt.plot([df_predicted_vs_observed_test3.min().min(), df_predicted_vs_observed_test3.max().max()],
         [df_predicted_vs_observed_test3.min().min(), df_predicted_vs_observed_test3.max().max()],
         linestyle='--', color='red')
plt.xlabel('spi predicho')
plt.ylabel('spi observado')
plt.title('SPI predicho vs observado con datos de prueba y pcp_DJF, hgt_DJF, olr_DJF')
plt.tight_layout()
plt.grid(True)
plt.show()

In [ ]:
# Combinar predicciones con los años correspondientes
df_train_pred_rl3 = pd.DataFrame({'Year': X_train3.index.map(lambda i: y_df_anios.loc[i, 'Year']),
                              'spi_predicted': y3_pred_train_df.squeeze()})

df_test_pred_rl3 = pd.DataFrame({'Year': X_test3.index.map(lambda i: y_df_anios.loc[i, 'Year']),
                             'spi_predicted': y2_pred_test_df.squeeze()})

# Unir todo y ordenar cronológicamente
df_pred_total_rn3 = pd.concat([df_train_pred_rl3, df_test_pred_rl3])
df_pred_total_rn3 = df_pred_total_rn3.sort_values('Year').reset_index(drop=True)


plt.figure(figsize=(10,5))
plt.plot(df['Year'], df['spi'], label='Observado', marker='o')
#sns.lineplot(x='Year',y='spi', data=df, markers='o')
plt.plot(df_pred_total_rn1['Year'], df_pred_total_rn1['spi_predicted'], label='Predicho AD pcp_DJF, olr_DJF, swvl1_DJF', marker='s')
plt.plot(df_pred_total_rn2['Year'], df_pred_total_rn2['spi_predicted'], label='Predicho AD hgt_DJF, skt_DJF, sst_mean_DJF', marker='s')
plt.plot(df_pred_total_rn3['Year'], df_pred_total_rn3['spi_predicted'], label='Predicho AD pcp_DJF, hgt_DJF, olr_DJF', marker='s')
plt.legend()
plt.xlabel('Año')
plt.ylabel('SPI')
plt.title('Serie temporal observada vs predicha')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plot_tree(dtr3.estimators_[0],
               feature_names = x3_scaled_df.columns,
               filled=True,
               rounded=True)
plt.show()

In [ ]:
df_pi

### **<font color="SeaGreen"> Paso 4: Implementar el/los modelos disponibles. </font>**


Para los árboles de decisión clasificatorias:https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html



`sklearn.ensemble.ExtraTreesClassifier(n_estimators=100, *, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=False, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None, monotonic_cst=None)`




In [ ]:
# Para implementar las maquinas de soporte vectorial para clasificacion, 
# primero vamos a crear una etiqueda de salida.

def categorizar_spi(spi):
    if spi <= -0.5 : 
        return 'sequia'
    else:
        return 'sin_sequia'

In [ ]:
df['sequia'] = df['spi'].apply(categorizar_spi)
df.head()

In [ ]:
df['sequia'].value_counts()

In [ ]:
# Definimos nuestra variable dependiente para categorizar.

y_cat = df[['sequia']].copy()

In [ ]:
X_train1c, X_test1c, y_train1c, y_test1c = train_test_split(x1_scaled_df, y_cat, test_size=0.4, random_state=43)

X_train2c, X_test2c, y_train2c, y_test2c = train_test_split(x2_scaled_df, y_cat, test_size=0.4, random_state=43)

X_train3c, X_test3c, y_train3c, y_test3c = train_test_split(x3_scaled_df, y_cat, test_size=0.4, random_state=43)

In [ ]:
# Definimos el modelo
dtc1 =  ExtraTreesClassifier(random_state=100, n_estimators=6, max_depth=3)

# Ajustamos el modelo
dtc1.fit(X_train1c,y_train1c.values.flatten())

# Hacemos las predicciones 
yc1_pred_train = dtc1.predict(X_train1c)
yc1_pred_test = dtc1.predict(X_test1c)


In [ ]:
# Definimos el modelo
dtc2 = ExtraTreesClassifier(random_state=100, n_estimators=6, max_depth=3)

# Ajustamos el modelo
dtc2.fit(X_train2c,y_train2c.values.flatten())

# Hacemos las predicciones 
yc2_pred_train = dtc2.predict(X_train2c)
yc2_pred_test = dtc2.predict(X_test2c)


In [ ]:
# Definimos el modelo
dtc3 = ExtraTreesClassifier(random_state=100, n_estimators=6, max_depth=3)

# Ajustamos el modelo
dtc3.fit(X_train3c,y_train3c.values.flatten())

# Hacemos las predicciones 
yc3_pred_train = dtc3.predict(X_train3c)
yc3_pred_test = dtc3.predict(X_test3c)


In [ ]:
perf_index={'predictores':[],'f1':[],'falsas alarmas':[]}
ML_method=[]
df_pic=pd.DataFrame(perf_index, index=ML_method)
df_pic = df_pic.rename_axis('ML methods')

In [ ]:
yc1_pred_train_df = pd.DataFrame(yc1_pred_train,columns=['sequia_pred'])
yc1_pred_test_df = pd.DataFrame(yc1_pred_test,columns=['sequia_pred'])

Etiquetas = ['sequia','sin_sequia']
confusion_1_train = confusion_matrix(y_train1c['sequia'],yc1_pred_train_df['sequia_pred'])
ConfusionMatrixDisplay(confusion_matrix=confusion_1_train, display_labels=Etiquetas).plot()

f1_score_train1 = f1_score(yc1_pred_train_df['sequia_pred'], y_train1c['sequia'],pos_label='sequia')
precision_train1 = precision_score(yc1_pred_train_df['sequia_pred'], y_train1c['sequia'],pos_label='sequia')

row_list=[', '.join(x_prep_1.columns),f1_score_train1,precision_train1]
df_pic.loc['DTC entrenamiento 1']=row_list

confusion_1_test = confusion_matrix(y_test1c['sequia'],yc1_pred_test_df['sequia_pred'])
ConfusionMatrixDisplay(confusion_matrix=confusion_1_test, display_labels=Etiquetas).plot()

f1_score_test1 = f1_score(yc1_pred_test_df['sequia_pred'], y_test1c['sequia'],pos_label='sequia')
precision_test1 = precision_score(yc1_pred_test_df['sequia_pred'], y_test1c['sequia'],pos_label='sequia')

row_list=[', '.join(x_prep_1.columns),f1_score_test1,precision_test1]
df_pic.loc['DTC prueba 1']=row_list
print(df_pic)

In [ ]:
plt.figure(figsize=(10,10))
plot_tree(dtc1.estimators_[0],
               feature_names = x1_scaled_df.columns,
               filled=True,
               rounded=True)
plt.show()

In [ ]:
print(classification_report(y_train1c['sequia'], yc1_pred_train_df['sequia_pred']))
print(classification_report(y_test1c['sequia'], yc1_pred_test_df['sequia_pred']))

In [ ]:
yc2_pred_train_df = pd.DataFrame(yc2_pred_train,columns=['sequia_pred'])
yc2_pred_test_df = pd.DataFrame(yc2_pred_test,columns=['sequia_pred'])

Etiquetas = ['sequia','sin_sequia']
confusion_2_train = confusion_matrix(y_train2c['sequia'],yc2_pred_train_df['sequia_pred'])
ConfusionMatrixDisplay(confusion_matrix=confusion_2_train, display_labels=Etiquetas).plot()

f1_score_train2 = f1_score(yc2_pred_train_df['sequia_pred'], y_train2c['sequia'],pos_label='sequia')
precision_train2 = precision_score(yc2_pred_train_df['sequia_pred'], y_train2c['sequia'],pos_label='sequia')

row_list=[', '.join(x_prep_2.columns),f1_score_train2,precision_train2]
df_pic.loc['DTC entrenamiento 2']=row_list


confusion_2_test = confusion_matrix(y_test2c['sequia'],yc2_pred_test_df['sequia_pred'])
ConfusionMatrixDisplay(confusion_matrix=confusion_2_test, display_labels=Etiquetas).plot()

f1_score_test2 = f1_score(yc2_pred_test_df['sequia_pred'], y_test2c['sequia'],pos_label='sequia')
precision_test2 = precision_score(yc2_pred_test_df['sequia_pred'], y_test2c['sequia'],pos_label='sequia')

row_list=[', '.join(x_prep_2.columns),f1_score_test2,precision_test2]
df_pic.loc['DTC prueba 2']=row_list
print(df_pic)

In [ ]:
plt.figure(figsize=(10,10))
plot_tree(dtc2.estimators_[0],
               feature_names = x2_scaled_df.columns,
               filled=True,
               rounded=True)
plt.show()

In [ ]:
yc3_pred_train_df = pd.DataFrame(yc3_pred_train,columns=['sequia_pred'])
yc3_pred_test_df = pd.DataFrame(yc3_pred_test,columns=['sequia_pred'])

Etiquetas = ['sequia','sin_sequia']
confusion_3_train = confusion_matrix(y_train3c['sequia'],yc3_pred_train_df['sequia_pred'])
ConfusionMatrixDisplay(confusion_matrix=confusion_3_train, display_labels=Etiquetas).plot()

f1_score_train3 = f1_score(yc3_pred_train_df['sequia_pred'], y_train3c['sequia'],pos_label='sequia')
precision_train3 = precision_score(yc3_pred_train_df['sequia_pred'], y_train3c['sequia'],pos_label='sequia')

row_list=[', '.join(x_prep_3.columns),f1_score_train3,precision_train3]
df_pic.loc['DTC entrenamiento 3']=row_list


confusion_3_test = confusion_matrix(y_test3c['sequia'],yc3_pred_test_df['sequia_pred'])
ConfusionMatrixDisplay(confusion_matrix=confusion_3_test, display_labels=Etiquetas).plot()

f1_score_test3 = f1_score(yc3_pred_test_df['sequia_pred'], y_test3c['sequia'],pos_label='sequia')
precision_test3 = precision_score(yc3_pred_test_df['sequia_pred'], y_test3c['sequia'],pos_label='sequia')

row_list=[', '.join(x_prep_3.columns),f1_score_test3,precision_test3]
df_pic.loc['DTC prueba 3']=row_list
print(df_pic)

In [ ]:
plt.figure(figsize=(10,10))
plot_tree(dtc3.estimators_[0],
               feature_names = x3_scaled_df.columns,
               filled=True,
               rounded=True)
plt.show()

---
<a name='ej-1'></a>
### **<font color="DodgerBlue"> Ejercicio 1 :   </font>**

<font color="DarkBlue">  Obten el modelo de AD regresiva del PM10 considerando las variables predictoras más importantes (recordar cuales son). Despues, crea la categorizacion de la calidad del aire, si el PM10 es mayor o igual a 35 ppm, es mala, en otro caso es buena. Luego, crea el modelo de AD categorica. 

---

<div class="alert alert-block alert-warning">
    <b>Nota: <b> En este caso tambien deben estandarizar la variable dependiente (PM10), y despues de hacer las predicciones, deben des-estandarizar los datos `.inverse_transform(.reshape(-1, 1))`
</div>
